In [ ]:
# Cell 1: Setup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import glob
from datetime import datetime

# Configure plotting
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = '#1e1e1e'
plt.rcParams['axes.facecolor'] = '#2d2d2d'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['text.color'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'

# Settings
LOG_DIR = Path("tools")  # Changed to tools folder
NUM_LOGS_TO_PLOT = 1  # Set to 1 for single log, or higher for overlay

print(f"[INFO] Looking for logs in: {LOG_DIR.resolve()}")
print(f"[INFO] Will plot {NUM_LOGS_TO_PLOT} most recent log(s)")

In [ ]:

# Cell 2: Helper Functions
def get_latest_log(log_dir: Path) -> Optional[Path]:
    """Get the most recent CSV log file."""
    if not log_dir.exists():
        print(f"[ERROR] Directory does not exist: {log_dir}")
        return None
    
    csv_files = list(log_dir.glob("balance_*.csv"))
    if not csv_files:
        print(f"[WARN] No CSV files found in {log_dir}")
        return None
    
    # Sort by modification time (most recent first)
    latest = max(csv_files, key=lambda f: f.stat().st_mtime)
    print(f"[INFO] Found {len(csv_files)} log files")
    print(f"[INFO] Most recent: {latest.name}")
    return latest

def get_latest_logs(log_dir: Path, n: int) -> List[Path]:
    """Get n most recent CSV log files."""
    if not log_dir.exists():
        print(f"[ERROR] Directory does not exist: {log_dir}")
        return []
    
    csv_files = list(log_dir.glob("balance_*.csv"))
    if not csv_files:
        print(f"[WARN] No CSV files found in {log_dir}")
        return []
    
    # Sort by modification time and take n most recent
    sorted_files = sorted(csv_files, key=lambda f: f.stat().st_mtime, reverse=True)
    return sorted_files[:n]

In [ ]:

# Cell 3: Load Data
log_file = None
log_list = []

if NUM_LOGS_TO_PLOT == 1:
    log_file = get_latest_log(LOG_DIR)
    
    if log_file:
        print(f"\n[LOAD] Loading: {log_file.name}")
        try:
            df = pd.read_csv(log_file)
            
            # Check if file has data
            if df.empty:
                print(f"[ERROR] CSV file is empty: {log_file}")
            else:
                print(f"[INFO] Loaded {len(df)} rows")
                print(f"[INFO] Columns: {list(df.columns)}")
                
                # Metadata Extraction with error handling
                try:
                    gains = {
                        "Kp": df["kp_balance"].iloc[0] if "kp_balance" in df.columns else float("nan"),
                        "Ki": df["ki_balance"].iloc[0] if "ki_balance" in df.columns else float("nan"),
                        "Kd": df["kd_balance"].iloc[0] if "kd_balance" in df.columns else float("nan")
                    }
                    reason = df["reason"].iloc[0] if "reason" in df.columns else "unknown"
                    print(f"[INFO] Gains: {gains}")
                    print(f"[INFO] End Reason: {reason}")
                except Exception as e:
                    print(f"[WARN] Could not extract metadata: {e}")
                    gains = {"Kp": 0, "Ki": 0, "Kd": 0}
                    reason = "unknown"
                
                # Normalize Time to start at 0
                if "time_s" in df.columns:
                    df["t"] = df["time_s"] - df["time_s"].iloc[0]
                else:
                    print("[WARN] No 'time_s' column found, creating dummy time")
                    df["t"] = range(len(df))
                
        except pd.errors.EmptyDataError:
            print(f"[ERROR] CSV file is empty: {log_file}")
        except Exception as e:
            print(f"[ERROR] Failed to load CSV: {e}")
    else:
        print(f"[ERROR] No log files found in {LOG_DIR}")
else:
    log_files = get_latest_logs(LOG_DIR, NUM_LOGS_TO_PLOT)
    if log_files:
        print(f"\n[LOAD] Loading {len(log_files)} logs: {[p.name for p in log_files]}")
        for f in log_files:
            try:
                dfi = pd.read_csv(f)
                if not dfi.empty:
                    dfi["t"] = dfi["time_s"] - dfi["time_s"].iloc[0] if "time_s" in dfi.columns else range(len(dfi))
                    log_list.append((f.name, dfi))
            except Exception as e:
                print(f"[WARN] Skipping {f.name}: {e}")
        
        if log_list:
            df = log_list[0][1]  # Use first for single-file plots
        else:
            print("[ERROR] No valid log files loaded")
    else:
        print(f"[ERROR] No log files found in {LOG_DIR}")


In [ ]:

# Cell 4: Plot 1 - System States (Angles)
if 'df' in locals() and not df.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Pendulum Angle
    if "pendulum_deg" in df.columns:
        ax1.plot(df["t"], df["pendulum_deg"], label="Pendulum Angle", color="tab:blue", linewidth=1.5)
    ax1.axhline(0, color="white", linestyle="--", alpha=0.3)
    ax1.set_ylabel("Angle (deg)")
    ax1.set_title(f"Pendulum Stability")
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Base Angle (Motor Position)
    if "base_deg" in df.columns:
        ax2.plot(df["t"], df["base_deg"], label="Base/Motor Angle", color="tab:orange", linewidth=1.5)
    ax2.set_ylabel("Angle (deg)")
    ax2.set_xlabel("Time (s)")
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("[ERROR] No data loaded. Cannot plot System States.")


In [ ]:

# Cell 5: Plot 2 - Control Effort & Phase Space
if 'df' in locals() and not df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Control Output (Hz)
    if "control_output" in df.columns:
        ax1.plot(df["t"], df["control_output"], color="tab:green", alpha=0.8, linewidth=1.5)
        ax1.set_title("Control Effort (Stepper Hz)")
        ax1.set_xlabel("Time (s)")
        ax1.set_ylabel("Steps/Sec (Hz)")
        ax1.grid(True, alpha=0.3)
    
    # Phase Portrait
    if "pendulum_deg" in df.columns and "pendulum_vel" in df.columns:
        ax2.plot(df["pendulum_deg"], df["pendulum_vel"], color="purple", alpha=0.6, linewidth=1)
        ax2.plot(df["pendulum_deg"].iloc[0], df["pendulum_vel"].iloc[0], 'go', label="Start")
        ax2.plot(df["pendulum_deg"].iloc[-1], df["pendulum_vel"].iloc[-1], 'rx', label="End")
        ax2.set_title("Phase Portrait (Pendulum)")
        ax2.set_xlabel("Angle (deg)")
        ax2.set_ylabel("Velocity (deg/s)")
        ax2.axhline(0, color="white", alpha=0.2)
        ax2.axvline(0, color="white", alpha=0.2)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("[ERROR] No data loaded. Cannot plot Control Effort.")


In [ ]:

# Cell 6: Summary Statistics
if 'df' in locals() and not df.empty:
    print("\n=== SUMMARY STATISTICS ===")
    print(f"Duration: {df['t'].iloc[-1]:.2f} seconds")
    print(f"Samples: {len(df)}")
    
    if "pendulum_deg" in df.columns:
        print(f"Pendulum RMS: {df['pendulum_deg'].std():.2f}°")
        print(f"Pendulum Max: {df['pendulum_deg'].abs().max():.2f}°")
    
    if "base_deg" in df.columns:
        print(f"Motor Range: {df['base_deg'].max() - df['base_deg'].min():.2f}°")
    
    if "control_output" in df.columns:
        print(f"Control RMS: {df['control_output'].std():.2f} Hz")
    
    if 'gains' in locals():
        print(f"Gains: Kp={gains['Kp']}, Ki={gains['Ki']}, Kd={gains['Kd']}")
    
    if 'reason' in locals():
        print(f"End Reason: {reason}")
